Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48817 non-null  float64
 1   bedrooms              48817 non-null  int64  
 2   created               48817 non-null  object 
 3   description           47392 non-null  object 
 4   display_address       48684 non-null  object 
 5   latitude              48817 non-null  float64
 6   longitude             48817 non-null  float64
 7   price                 48817 non-null  int64  
 8   street_address        48807 non-null  object 
 9   interest_level        48817 non-null  object 
 10  elevator              48817 non-null  int64  
 11  cats_allowed          48817 non-null  int64  
 12  hardwood_floors       48817 non-null  int64  
 13  dogs_allowed          48817 non-null  int64  
 14  doorman               48817 non-null  int64  
 15  dishwasher         

In [15]:
df.sample(4)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
39244,1.0,1,2016-06-20 18:50:54,This building is a new rental residence rising...,Washington Ave.,40.6938,-73.9669,3375,163 Washington Ave.,low,1,1,1,1,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
23695,1.0,0,2016-06-06 03:23:38,Beautiful spacious apartment literally right i...,W 47 St.,40.7587,-73.9838,2100,150 W 47 St.,medium,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48038,1.0,1,2016-04-13 02:25:01,Large 1 bedroom in new modern luxury building ...,Classon Avenue,40.6824,-73.9586,2900,535 Classon Avenue,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36455,2.0,2,2016-05-10 05:49:00,This apartment is newly renovated featuring: b...,East 9th Street,40.7258,-73.9798,3450,610 East 9th Street,medium,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Engineer two new features

In [19]:
# create 'amenities' feature
df['amenities'] = df['elevator'] + df['balcony'] + df['dining_room'] + df['fitness_center'] + df['high_speed_internet'] + df['laundry_in_unit'] + df['swimming_pool'] + df['pre-war'] + df['dishwasher'] + df['dogs_allowed']

In [21]:
# create bed+bath feature
df['bed_bath'] = df['bathrooms'] + df['bedrooms']

In [23]:
# check to see if the features were added to the dataframe
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space',
       'amenities', 'bed_bath'],
      dtype='object')

##  Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test

In [27]:
train = df[df['created'] < '2016-06-01']
test = df[df['created'] >= '2016-06-01']

In [28]:
# check the number of observations in each set
train.shape, test.shape

((31844, 36), (16973, 36))

## Fit a linear regression model with two new  features

In [34]:
# Declare target variable and feature matrix
target = ['price']
features = ['amenities', 'bed_bath']

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [31]:
# import linear regression from sklearn
from sklearn.linear_model import LinearRegression

In [32]:
# Instantiate the model
model = LinearRegression()

In [35]:
# fit the model to the training dataset
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
# Predict the y-values on the testing set
y_pred = model.predict(X_test)

In [42]:
 y_pred[:5]

array([[4438.26627575],
       [3917.32802471],
       [2535.6649891 ],
       [2949.4133364 ],
       [6821.07208964]])

In [44]:
y_test[:5]

,price
0,3000
1,5465
11,1675
14,3050
24,7400


## The model's coefficients and intercept

In [45]:
# intercept
model.intercept_

array([1013.58395979])

In [46]:
# coefficients
model.coef_

array([[206.87417365, 761.04051466]])

## Regression metrics RMSE, MAE, and  R2 , for both the train and test data

In [49]:
# MAE: On average, by how much value are we missing the mark?
from sklearn import metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
print(f'On average, our model errs by {round(mae)} dollars per apartment')
mae

On average, our model errs by 864.0 dollars per apartment


863.672962278647

In [53]:
# RMSE
rmse = metrics.mean_squared_error(y_test, y_pred)
rmse

1653082.774033967

In [52]:
# A higher R2 score indicates more variance explained
r2 = metrics.r2_score(y_test, y_pred)
round(r2,2)

0.47

## Assignment Questions

1.   How do I get the metrics for the training data? Do I need to use the model to predict the values in the training data first?



